In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import ee
import io
import tqdm
import json
import fiona
import datetime
import requests
import urllib.request

import numpy as np
import xarray as xr
import pandas as pd
import rsfuncs as rs
import rasterio as rio
import geopandas as gp
import multiprocessing as mp

import matplotlib.pyplot as plt

from tqdm import tqdm
from affine import Affine
from datetime import timedelta
from rasterio import features, mask
from climata.usgs import DailyValueIO
from pandas.tseries.offsets import MonthEnd
from dateutil.relativedelta import relativedelta

ee.Initialize()

In [2]:
# Read catchments, reservoirs
gdf = gp.read_file("../shape/sierra_catchments.shp")
reservoirs_gdf = gp.read_file("../shape/reservoirs_grace.shp")

# Load rs data
data = rs.load_data()

# Define start / end
strstart = '2001-01-01'
strend = '2019-12-31'

startdate = datetime.datetime.strptime(strstart, "%Y-%m-%d")
enddate = datetime.datetime.strptime(strend, "%Y-%m-%d")


In [3]:
def get_q_out(stid, startdate = "1997-01-01",  enddate = "2020-01-01"):
    '''
    Fetch the daily discharge for a USGS gauge ID
    Input: stid (str) - the station id
    Output: qdf - (pd.DataFrame) - the daily streamflow data
    '''
    
    data = DailyValueIO(
        start_date=startdate, 
        end_date=enddate,
        station=stid,
        parameter="00060",
    )
    
    qs = []
    ds = []
    lats = []
    lons = []
    ids = []

    for series in data:
        values = []
        dates = []
        lats.append(series.latitude)
        lons.append(series.longitude)
        ids.append(series.site_code)

        for row in series.data:
            values.append(row.value)
            dates.append(row.date)

        qs.append(values)
        ds.append(dates)
    
    qdf = pd.DataFrame(qs[0], ds[0]) # convert cfs to cms
    qdf.columns= ['q_cfs']
    qdf['q_cms'] = qdf['q_cfs'] * 0.0283168
    
    return qdf


def get_Sres_out(stid):
    '''
    Fetch the monthly reservoir storage (af) for a CDEC reservoir ID
    Input: stid (str) - 3 letter station id (ex: ISB)
    Output: Sres - (pd.DataFrame) - the monthly reservoir levels (km^3)
    '''
    # Download Storage (SensorNums = 15) data by query str:
    start = datetime.datetime(1997, 1, 1)
    end = datetime.datetime(2020, 1, 31)
    dt_idx = pd.date_range(start,end, freq='M')

    url = '''https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations={}&SensorNums=15&dur_code=M&Start=1997-01-01&End=2020-01-01'''.format(stid)
    urlData = requests.get(url).content
    df = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
    
    storage = pd.to_numeric(df.VALUE)

    storage_sum = storage * 1.23348e-6 # acre ft to km^3
    Sres = pd.DataFrame(zip(dt_idx,storage_sum), columns = ['date',"Sres_outlet"])
    
    return rs.col_to_dt(Sres)


def get_wshed_Sres(gdf):
    '''
    Same as above, but filters by polygon, rather than id
    Input: gdf (gp.GeoDataFrame) - area defining AOI 
    Output: Sres - (pd.DataFrame) - the monthly reservoir levels (km^3)
    '''
    within_gdf = gdf.copy()

    # Download Storage (SensorNums = 15) data by query str:
    start = datetime.datetime(1997, 1, 1)
    end = datetime.datetime(2020, 1, 1)
    dt_idx = pd.date_range(start,end, freq='M')

    data = {}

    # Loop through stations 
    for i in within_gdf.ID:
        url = "https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations={}&SensorNums=15&dur_code=M&Start=1997-01-01&End=2020-01-01".format(i)
        urlData = requests.get(url).content
        df = pd.read_csv(io.StringIO(urlData.decode('utf-8')))

        if df.empty:
            pass
        else:
            data[i] = df

    # Concatenate 
    storage = []
    for k,v in data.items():
        storage.append(pd.to_numeric(data[k].VALUE, errors = "coerce"))

    storage_sum = np.nansum(np.column_stack(storage), axis = 1) * 1.23348e-6 # acre ft to km^3
    Sres = pd.DataFrame(zip(dt_idx,storage_sum), columns = ['date',"Sres"])
    Sres.set_index('date', inplace = True)
    
    return Sres


def get_lrm_swe(shppath):
    '''
    Given a path to a shapefile, compute the monthly SWE
    Input: (str) - path to shapefile
    Output: (pd.DataFrame) - monthly SWE 
    '''
    
    # Find SWE files
    data_dir = "../data/LRM_SWE_monthly"
    files = [os.path.join(data_dir,x) for x in os.listdir(data_dir) if x.endswith(".tif")]
    files.sort()

    # Read CVWS shapefile
    with fiona.open(shppath, "r") as shapefile:
        cvws_geom = [feature["geometry"] for feature in shapefile]

    # Read the files, mask nans, clip to CVWS, extract dates
    imdict = {}

    for i in tqdm(files[:]):
        date = datetime.datetime.strptime(i[-12:-4],'%Y%m%d')+ timedelta(days=-1) # Get the date 
        datestr = date.strftime('%Y%m%d') # Format date
        src = rio.open(i) # Read file
        src2 = rio.mask.mask(src, cvws_geom, crop=True) # Clip to shp 
        arr = src2[0] # read as array
        arr = arr.reshape(arr.shape[1], arr.shape[2]) # Reshape bc rasterio has a different dim ordering 
        arr[arr < 0 ] = np.nan # Mask nodata vals 
        imdict[datestr] = arr
        
    # Fill in the dates with no SWE with nans 
    dt_idx = pd.date_range(list(imdict.keys())[0], list(imdict.keys())[-1], freq = "M")

    all_dates = {}

    for i in dt_idx:
        date = i.strftime("%Y%m%d") 

        if date in imdict.keys():
            im = imdict[date]
        else:
            im = np.zeros_like(list(imdict.values())[0])
            im[im==0] = np.nan
        all_dates[date] = im

    # Stack all dates to 3D array
    cvws_swe = np.dstack(list(all_dates.values()))

    # Compute monthly sums
    swesums = []
    for i in range(cvws_swe.shape[2]):
        swesums.append(np.nansum(cvws_swe[:,:,i] *500**2 * 1e-9)) # mult by 2500m pixel area, convert m^3 to km^3

    swedf = pd.DataFrame(swesums,dt_idx)
    swedf.columns = ['swe_lrm']
    return swedf


def get_ssebop(shppath):
    '''
    Given a path to a shapefile, compute the monthly SSEBop ET
    Input: (str) - path to shapefile
    Output: (pd.DataFrame) - monthly SSEBop ET  
    '''
    
    files = [os.path.join("../data",x) for x in os.listdir("../data") if x.endswith("nc") if "SSEBOP" in x]
    ds = xr.open_dataset(files[0])
    
    gdf = gp.read_file(shppath)
    
    ds['catch'] = rasterize(gdf.geometry, ds['et'][0].coords)
    ssebop_masked = ds['et']*ds['catch']
    
    dt = pd.date_range(ds.time[0].values, ds.time[-1].values, freq = "MS")

    et= []
    for i in ssebop_masked:
        et.append(i.sum()* 1e-6) # m^2 to km^2 
        
    etdf = pd.DataFrame({'et': np.array(et)}, index = dt)
    etdf.columns = ["aet_ssebop"]
    etdf.set_index(etdf.index + MonthEnd(0), inplace = True)
    
    return etdf
    
def transform_from_latlon(lat, lon):
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    trans = Affine.translation(lon[0], lat[0])
    scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
    return trans * scale

def rasterize(shapes, coords, fill=np.nan, **kwargs):
    """Rasterize a list of (geometry, fill_value) tuples onto the given
    xray coordinates. This only works for 1d latitude and longitude
    arrays.
    """
    transform = transform_from_latlon(coords['lat'], coords['lon'])
    out_shape = (len(coords['lat']), len(coords['lon']))
    raster = features.rasterize(shapes, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float, **kwargs)
    return xr.DataArray(raster, coords=coords, dims=('lat', 'lon'))

In [4]:
for idx, x in enumerate(gdf[:].iterrows()):
    print(idx)
    print("****" * 15)
    row  = x[1]
    stn_id = row['stid']
    print(stn_id, row['catch_name'])
    
    # Get the outflow, to the reservoir or through the river
    if not stn_id.isdigit():
        outflow = get_Sres_out(stn_id)

    else:
        Qsout_daily = get_q_out(stn_id)
        outflow = pd.DataFrame(Qsout_daily['q_cms'].resample("M").sum()) * 1e-9 * 86400 # cms to km^3 / mon
        outflow.columns = ['qout_km3']

    # Get the within wshed reservoir data (dont double count outlet)
    catch = gp.read_file("../shape/catchment_{}.shp".format(stn_id))
    
    # Spatial join cdec reservoirs to supplied catchment 
    reservoirs = gp.read_file("../shape/reservoirs_grace.shp")
    within_catch = gp.sjoin(reservoirs, catch, how='inner', op='within')
    within_catch = within_catch[~within_catch['ID'].isin([stn_id])]
    
    # Handle cases with no reservoirs
    if len(within_catch) > 0:
        Sres = get_wshed_Sres(within_catch)
    else:
        Sres_arr = np.zeros_like(outflow.values)
        Sres = pd.DataFrame(Sres_arr,outflow.index)
        Sres.columns = ['Sres']
        
    # Get the SWE LRM RS data    
    catch_shp = "../shape/catchment_{}.shp".format(stn_id)
    lrm_swe = get_lrm_swe(catch_shp)
    
    # Get the SSEBop ET RS data
    ssebop_et = get_ssebop(catch_shp)

    # Get the RS data (EE)
    area = rs.gdf_to_ee_poly(gp.GeoDataFrame(gdf.loc[idx]).T)
    prism = rs.calc_monthly_sum(data['prism'], strstart, strend, area)
    modis_aet = rs.calc_monthly_sum(data['modis_aet'], strstart, strend, area)
    modis_pet = rs.calc_monthly_sum(data['modis_pet'], strstart, strend, area)
    tc_sm = rs.calc_monthly_mean(data['tc_sm'], strstart, strend, area)
    dmet_swe = rs.calc_monthly_mean(data['dmet_swe'],strstart, strend, area)
    
    prism.set_index(prism.index + MonthEnd(0), inplace = True)
    modis_aet.set_index(modis_aet.index + MonthEnd(0), inplace = True)
    modis_pet.set_index(modis_pet.index + MonthEnd(0), inplace = True)
    tc_sm.set_index(tc_sm.index + MonthEnd(0), inplace = True)
    dmet_swe.set_index(dmet_swe.index + MonthEnd(0), inplace = True)
    
    prism.columns = ['p_prism']
    modis_aet.columns = ['aet_modis']
    modis_pet.columns = ['pet_modis']
    tc_sm.columns = ['tc_sm']
    dmet_swe.columns = ['dmet_swe']

    # Build the out dataframe 
    mdf = pd.concat([prism,ssebop_et,modis_aet, modis_pet, tc_sm, dmet_swe, lrm_swe,Sres,outflow],axis = 1)
    
    # Write
    outfn = "../data/RS_catchdat/catch_{}.csv".format(stn_id)
    
    mdf.to_csv(outfn)
    

0
************************************************************
NHG CALAVERAS R


100%|██████████| 116/116 [00:00<00:00, 148.43it/s]


processing:
OREGONSTATE/PRISM/AN81m/189501



processing:
MODIS/006/MOD16A2/2001_01_01



processing:
MODIS/006/MOD16A2/2001_01_01


...

processing:
OREGONSTATE/PRISM/AN81m/189501



processing:
MODIS/006/MOD16A2/2001_01_01



processing:
MODIS/006/MOD16A2/2001_01_01



processing:
IDAHO_EPSCOR/TERRACLIMATE/195801



processing:
NASA/ORNL/DAYMET_V3/19800101
